# Análise de lojas de instrumentos musicais - Parte 01 Web Scraping
<hr>

## **0.0 Objetivo e Introdução**

O objetivo deste projeto é analisar os dados de instrumentos musicais extraídos de três diferentes sites. A **parte 01** vai focar apenas na parte de Web Scraping e a **parte 02** na análise exploratória e clusterização.
<hr>

**Objetivo da Parte 01** <br>
Extrair dados dos instrumentos musicais e exportar estas informações para um arquivo Excel. Além da linguagem **Python**, para extrair os dados dos sites serão utilizadas as bibliotecas **Beautiful Soup** e também **Selenium** para suporte em alguns casos. Para facilitar a exportação e análise posterior, vamos transformar os dados em um Dataframe utilizando **Pandas**.

**Importante!** <br>
Antes de começar a extrair os dados do site, adicionei `robots.txt` ao final da URL da página inicial para checar se não existiam restrições para a extração dos dados. Como não haviam, podemos seguir.

**Sites**
1. [Tango](https://www.tangomusic.com.br/)
2. [Made in Brazil](https://www.madeinbrazil.com.br/)
3. [Barra Music](https://www.barramusic.com.br/)


## 1.0 Estratégia

**Para conseguir extrair os dados decidi quebrar o problema em três partes menores:**
1. Dentro da página do instrumento, ou seja a URL do item, conseguir extrair todas as informações desejadas (Nome, marca, preço, etc.)
2. Dentro da página de uma categoria (baixo, guitarra, violão), conseguir extrair as URLs de todos os produtos
3. Conseguir percorrer todas as páginas de uma categoria qualquer, pois para determinada categoria de instrumento podem existir mais de uma página com itens.

**Serão criadas duas funções para cada site** <br>
`get_item` --> Extrair todas as informações de cada URL (item) <br>
`get_urls` --> Percorrer todas as páginas e extrair todas as URLs

<hr>

**Informações que serão extraídas:** <br>
Instrumentos: **Contrabaixo, Guitarra e Violão**

1. Nome
2. Marca
3. Preço total
4. Máximo número de parcelas
5. Preço com desconto a vista
6. Link da imagem
7. Descrição
8. Loja

## 2.0 Imports

Importanto as bibliotecas necessárias. <br>
**Importante!** Para conseguir rodar o Selenium é necessário instalar o webdriver compatível com seu navegador, neste notebook estou utilizando Google Chrome rodando no Windows. Sem o webdriver não será possíver rodar o código mais adiante, na parte onde criei a variável "driver".

In [1]:
#Acessar as páginas 
import requests

#Beautiful Soup para Web Scraping
from bs4 import BeautifulSoup as bs

#Pandas para criar o DataFrame
import pandas as pd

#Regex
import re

#Funções matemáticas
import math

#Para atrasar a execução de alguns comandos
import time

#Selenium para conseguirmos lidar com animações nas páginas
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Impede que o Selenium abra uma nova janela
op = webdriver.ChromeOptions()
op.add_argument("headless")

## 3.0 Tango Music

### 3.1 Informações do item

In [2]:
def tango_get_item(url_list):
    
    '''Recebe uma lista de urls de itens do site Tango e retorna uma lista de dicionários 
    com as informações de cada instrumento. O resultado pode ser facilmente convertido em um DataFrame'''
    
    item_dict_list = []
    
    for url in url_list:
        
        #Carregando a página e convertendo para um objeto Beautiful Soup  
        r = requests.get(url)
        instrument = bs(r.content)

        item_dict = {}

        #1 de 8 - Nome
        item_dict["Name"] = instrument.find("h1", {"class":"prod_title color1 hidden-sm hidden-xs"}).div.get_text()

        #2 de 8 - Marca
        item_dict["Brand"] = instrument.find("div", {"class":"prod_info"}).a.get_text()

        #3 de 8 - Preço total
        #Checando se existe preço para o produto
        if instrument.find("strong", {"class":"skuBestPrice"}) == None:
            item_dict["Total_Price"] = "Sku_Indisponível"

        else:
            total_price = instrument.find("strong", {"class":"skuBestPrice"}).get_text()
            total_price = total_price.replace("R$","").replace('.','').replace(',','.').strip()
            item_dict["Total_Price"] = total_price

        #4 de 8 Número máximo de parcelas
        #Checando se a informação existe para o produto
        if instrument.find("label", {"class":"skuBestInstallmentNumber"}) == None:
            item_dict["Number_Installments"] = "NaN"

        else:
            item_dict["Number_Installments"] = instrument.find("label", {"class":"skuBestInstallmentNumber"}).get_text()

        #5 de 8 Preço à vista -- Este site não oferece desconto à vista =(
        item_dict["Cash_Price"] = "NaN"

        #6 de 8 URL da imagem do produto
        item_dict["Image_Url"] = instrument.find("a", {"class":"image-zoom"}).get("href")

        #7 de 8 Descrição do produto
        item_dict["Description"] = instrument.find("div", {"class":"productDescription"}).get_text()

        #8 de 8 Nome da loja
        item_dict["Store"] = instrument.find("h1", {"class":"logo"}).get_text()

        #Adicionando o dicionário à lista
        item_dict_list.append(item_dict)
        
    return item_dict_list

### 3.2 Buscando as URLs

**A parte abaixo teve dois desafios interessantes** <br>

**1) A classe do html varia de acordo com a categoria do instrumento.** <br>
Na linha 19, foi preciso deixar a categoria como variável para que o código funcionasse para qualquer instrumento. Por exemplo, caso deixasse `cordas` como padrão, ao passar a url da página de bateria teríamos um erro já que baterias ficam na categoria `percussão` 

**2) A atualização depois do carregamento da página.** <br>
Abrindo a página apenas usando `request` como foi feito nas linhas acima sempre resultava com os instrumentos da primeira página. Isso acontecia porque, mesmo passando uma nova página na URL, primeiro a página carrega os instrumentos do primeiro resultado para em seguida atualizar com os valores reais da página. Tentei algumas soluções com o `time.sleep` sozinho e acabou não funcionando. 

A solução foi utilizar o pacote **Selenium** para primeiro carregar a página, esperar **5 segundos** para atualizar as informações e só então enviar o html para o Beautiful Soup realizar o Scrapping.

In [3]:
def tango_get_urls(pages_url):
    
    '''Recebe uma url de categoria como 'https://www.tangomusic.com.br/teclado/teclado?PS=40' 
    e retorna as urls dos produtos de todas as páginas daquela categoria'''
    
    url_list = []
    
    #Estrutura da url
    pages_url = pages_url + "#01"
    
    #Categoria do instrumento
    category = pages_url.split("/")[-2]

    #Carregando a página e convertendo para um objeto Beautiful Soup
    r = requests.get(pages_url)
    new_url = bs(r.content)

    #Número total de instrumentos
    total_instruments = new_url.find("h4", {"class":category + " even"}).get_text()

    #Usando regex para pegar o número entre parênteses
    total_instruments = re.search(r'\((.*?)\)',total_instruments).group(1)

    #Temos 40 instrumentos por página
    total_pages = math.ceil(int(total_instruments)/40)

    print("Total instruments: {}".format(total_instruments))
    print("Number of pages: {}".format(total_pages))

    #Buscando todas urls
    for i in range(1, total_pages+1):

        #Estrutura da url para diferentes páginas
        pages_url = pages_url[:-1] + str(i)

        #Carrega a página
        driver = webdriver.Chrome(executable_path ="chromedriver", options = op)
        driver.get(pages_url)   

        #Espera 5 segundos para carregar completamente
        time.sleep(5)
    
        #Código fonte da página que será enviado ao Beautiful Soup
        page_source = driver.page_source

        #Converte para um objeto Beautiful Soup
        instrument_url = bs(page_source, "lxml")

        #Imprime a página atual
        print(pages_url)

        #Lista de instrumentos da página atual
        for i in instrument_url.find_all("div", {"class":"prod_wrapper"}):
            url_list.append(i.a.get("href"))

    print("-----------------")
    
    return url_list

### 3.3 Buscando todos os baixos, guitarras e violões e convertendo para um DataFrame
Com as funções definidas, vamos aplicá-las para conseguir extrair os dados! 

In [4]:
#Buscando as urls
tango_bass_urls = tango_get_urls("https://www.tangomusic.com.br/cordas/contrabaixo?PS=40")
tango_acoustic_guitar_urls = tango_get_urls("https://www.tangomusic.com.br/cordas/violao?PS=40")
tango_electric_guitar_urls = tango_get_urls("https://www.tangomusic.com.br/cordas/guitarra?PS=40")

#Buscando as informações dos itens
tango_bass_info = tango_get_item(tango_bass_urls)
tango_acoustic_guitar_info = tango_get_item(tango_acoustic_guitar_urls)
tango_electric_guitar_info = tango_get_item(tango_electric_guitar_urls)

#Criando o DataFrame
tango_bass_df = pd.DataFrame(tango_bass_info)
tango_acoustic_guitar_df = pd.DataFrame(tango_acoustic_guitar_info)
tango_electric_guitar_df = pd.DataFrame(tango_electric_guitar_info)

Total instruments: 95
Number of pages: 3
https://www.tangomusic.com.br/cordas/contrabaixo?PS=40#01
https://www.tangomusic.com.br/cordas/contrabaixo?PS=40#02
https://www.tangomusic.com.br/cordas/contrabaixo?PS=40#03
-----------------
Total instruments: 274
Number of pages: 7
https://www.tangomusic.com.br/cordas/violao?PS=40#01
https://www.tangomusic.com.br/cordas/violao?PS=40#02
https://www.tangomusic.com.br/cordas/violao?PS=40#03
https://www.tangomusic.com.br/cordas/violao?PS=40#04
https://www.tangomusic.com.br/cordas/violao?PS=40#05
https://www.tangomusic.com.br/cordas/violao?PS=40#06
https://www.tangomusic.com.br/cordas/violao?PS=40#07
-----------------
Total instruments: 210
Number of pages: 6
https://www.tangomusic.com.br/cordas/guitarra?PS=40#01
https://www.tangomusic.com.br/cordas/guitarra?PS=40#02
https://www.tangomusic.com.br/cordas/guitarra?PS=40#03
https://www.tangomusic.com.br/cordas/guitarra?PS=40#04
https://www.tangomusic.com.br/cordas/guitarra?PS=40#05
https://www.tangomu

In [5]:
#Checando por valores duplicados
dfs = [tango_bass_df, tango_acoustic_guitar_df, tango_electric_guitar_df]

for i in dfs:
    print(i.shape)
    print(i.duplicated().value_counts(), "\n")

(95, 8)
False    95
dtype: int64 

(274, 8)
False    274
dtype: int64 

(210, 8)
False    210
dtype: int64 



In [6]:
#Juntando os três DataFrames em um só

#Ajustando o tamanho das colunas
pd.set_option('max_colwidth', 40)

#Juntando os DataFrames e imprimindo uma amostra dos dados
tango_df = pd.DataFrame()
tango_df = tango_df.append([tango_bass_df, tango_electric_guitar_df, tango_acoustic_guitar_df])

print(tango_df.shape)

tango_df.sample(5)

(579, 8)


,Name,Brand,Total_Price,Number_Installments,Cash_Price,Image_Url,Description,Store
168,VIOLAO TAGIMA MONTANA TUNER BK - PRETO,TAGIMA,1089.00,12,NaN,https://tangomusic.vteximg.com.br/ar...,VIOLÃO AÇO TUNERBRAÇO / MahoganyTAMP...,Tango Music
121,GUITARRA STRINBERG CLG55 BK,STRINBERG,1099.00,12,NaN,https://tangomusic.vteximg.com.br/ar...,Especificações Técnicas:- Corpo: Bas...,Tango Music
144,GUITARRA TAGIMA T 635 CLASSIC ESCALA...,TAGIMA,1539.00,12,NaN,https://tangomusic.vteximg.com.br/ar...,T-635 Guitarra TagimaEspecificações:...,Tango Music
41,GUITARRA GIBSON LES PAUL JUNIOR VINT...,GIBSON,14115.00,12,NaN,https://tangomusic.vteximg.com.br/ar...,Gibson Les Paul Junior Vintage Tobac...,Tango Music
38,CONTRABAIXO 4C FENDER STANDARD DIMEN...,FENDER,6240.00,12,NaN,https://tangomusic.vteximg.com.br/ar...,A série Standard recebe seu primeiro...,Tango Music


## 4.0 Made in Brazil 

### 4.1 Informações do item

**Aqui tivemos uma situação interessante com as imagens.** <br>

Extraindo diretamente da página do instrumento resultava em uma imagem bastante pequena. Pela própria url `https://madeinbrazil.fbitsstatic.net/img/p/contrabaixo-4-cordas-tbm-4-serie-classic-tagima-preto-bk-69519/256002-1.jpg?w=120&h=120&v=no-change` é possível ver que temos a largura (width, w) e altura (height, h) por padrão igual a 120. 

No código simplesmente selecionei este pedaço da string e alterei os valores para 1000, resultando em uma imagem com tamanho maior! 

In [2]:
def made_in_brazil_get_item(url_list):
    
    '''Recebe uma lista de urls de itens do site Made in Brazil e retorna uma lista de dicionários 
    com as informações de cada instrumento. O resultado pode ser facilmente convertido em um DataFrame'''
    
    item_dict_list = []
    
    for url in url_list:
        
        #Carregando a página e convertendo para um objeto Beautiful Soup
        r = requests.get(url)
        instrument = bs(r.content)

        item_dict = {}

        #1 de 8 - Nome
        item_dict["Name"] = instrument.find("h1", {"class":"fbits-produto-nome prodTitle title"}).get_text()[2:]
        
        #2 de 8 - Marca
        #Checando se existe a informação de marca
        if instrument.find("div", {"class":"fbits-fabricante-logo logo-fabricante"}) == None:
            item_dict["Brand"] = "NaN"
        
        else:
            item_dict["Brand"] = instrument.find("div", {"class":"fbits-fabricante-logo logo-fabricante"}).a.img.get("title")

        #3 de 8 - Preço total
        total_price = instrument.find("div", {"class":"precoPor"}).get_text()
        total_price = total_price.replace("R$","").replace('.','').replace(',','.').strip()
        item_dict["Total_Price"] = total_price

        #4 de 8 Número máximo de parcelas
        item_dict["Number_Installments"] = instrument.find("span", {"class":"fbits-quantidadeParcelas"}).get_text()
       
        #5 de 8 Preço à vista
        cash_price = instrument.find("span", {"class":"fbits-boleto-preco"}).get_text()
        cash_price = cash_price.replace("R$","").replace('.','').replace(',','.').strip()
        item_dict["Cash_Price"] = cash_price

        #6 de 8 URL da imagem do produto
        image_url = instrument.find("div", {"class":"jcarousel"}).img.get("src").split("=")

        #Como a imagem extraída era muito pequena, vamos aumentar suas dimensões pela url
        image_url[1] = "1000&h"
        image_url[2] = "1000&v"

        #Checando se a imagem existe
        if "".join(image_url).split("/")[5] == "produto-nao-possui-foto-no-momento":
            item_dict["Image_Url"] = "NaN"

        else:
            item_dict["Image_Url"] = "".join(image_url)
        
        #7 de 8 Descrição do produto
        #Checando se existe descrição para o produto
        if instrument.find("div", {"class":"paddingbox"}) == None:
            item_dict["Description"] = "NaN"

        else:
            item_dict["Description"] = instrument.find("div", {"class":"paddingbox"}).get_text().replace("\n","")
            
        #8 de 8 Nome da loja
        item_dict["Store"] = instrument.find("i", {"class":"logotipo-footer"}).get_text()[:14]

        #Adicionando o dicionário à lista
        item_dict_list.append(item_dict)
        
    return item_dict_list

### 4.2 Buscando as URLs

Sem maiores desafios aqui, já que cada página tem uma url específica carregando apenas os instrumentos daquela exibição

In [3]:
def made_in_brazil_get_urls(pages_url):
    
    '''Recebe uma url de categoria como 'https://www.madeinbrazil.com.br/cordas-e-acessorios/contrabaixo' 
    e retorna as urls dos produtos de todas as páginas daquela categoria'''
    
    url_list = []
    
    #Estrutura da url
    pages_url = pages_url + "?pagina=01"
    
    #Carregando a página e convertendo para um objeto Beautiful Soup
    r = requests.get(pages_url)
    new_url = bs(r.content)

    #Número total de instrumentos
    total_instruments = new_url.find("div", {"class": "mostrando left fbits-info-bottom"}).span.get_text()

    #Usando regex para pegar apenas o número
    total_instruments = re.search(r'\d+',total_instruments).group(0)

    #Temos 40 instrumentos por página
    total_pages = math.ceil(int(total_instruments)/40)

    print("Total instruments: {}".format(total_instruments))
    print("Number of pages: {}".format(total_pages))

    #Buscando todas urls
    for i in range(1, total_pages+1):

        #Estrutura da url para diferentes páginas
        pages_url = pages_url[:-1] + str(i)

        #Carregando a página e convertendo para um objeto Beautiful Soup
        r = requests.get(pages_url)
        instrument_url = bs(r.content)
    
        #Imprime a página atual
        print(pages_url)
    
        #Lista de instrumentos da página atual
        for i in instrument_url.find_all("a", {"class":"spot-parte-um"}):
            url_list.append("https://www.madeinbrazil.com.br/" + i.get("href"))

    print("-----------------")
    
    return url_list

### 4.3 Buscando todos os baixos, guitarras e violões e convertendo para um DataFrame
Aplicando as funções para extrair os dados =) 

In [4]:
#Buscando as urls
made_in_brazil_bass_urls = made_in_brazil_get_urls("https://www.madeinbrazil.com.br/cordas-e-acessorios/contrabaixo")
made_in_brazil_acoustic_guitar_urls = made_in_brazil_get_urls("https://www.madeinbrazil.com.br/cordas-e-acessorios/violao-eletroacustico")
made_in_brazil_electric_guitar_urls = made_in_brazil_get_urls("https://www.madeinbrazil.com.br/cordas-e-acessorios/guitarra")

#Buscando as informações dos itens
made_in_brazil_bass_info = made_in_brazil_get_item(made_in_brazil_bass_urls)
made_in_brazil_acoustic_guitar_info = made_in_brazil_get_item(made_in_brazil_acoustic_guitar_urls)
made_in_brazil_electric_guitar_info = made_in_brazil_get_item(made_in_brazil_electric_guitar_urls)

#Criando o DataFrame
made_in_brazil_bass_df = pd.DataFrame(made_in_brazil_bass_info)
made_in_brazil_acoustic_guitar_df = pd.DataFrame(made_in_brazil_acoustic_guitar_info)
made_in_brazil_electric_guitar_df = pd.DataFrame(made_in_brazil_electric_guitar_info)

Total instruments: 16
Number of pages: 1
https://www.madeinbrazil.com.br/cordas-e-acessorios/contrabaixo?pagina=01
-----------------
Total instruments: 84
Number of pages: 3
https://www.madeinbrazil.com.br/cordas-e-acessorios/violao-eletroacustico?pagina=01
https://www.madeinbrazil.com.br/cordas-e-acessorios/violao-eletroacustico?pagina=02
https://www.madeinbrazil.com.br/cordas-e-acessorios/violao-eletroacustico?pagina=03
-----------------
Total instruments: 62
Number of pages: 2
https://www.madeinbrazil.com.br/cordas-e-acessorios/guitarra?pagina=01
https://www.madeinbrazil.com.br/cordas-e-acessorios/guitarra?pagina=02
-----------------


In [5]:
#Checando por valores duplicados
dfs = [made_in_brazil_bass_df, made_in_brazil_acoustic_guitar_df, made_in_brazil_electric_guitar_df]

for i in dfs:
    print(i.shape)
    print(i.duplicated().value_counts(), "\n")

(16, 8)
False    16
dtype: int64 

(84, 8)
False    84
dtype: int64 

(62, 8)
False    62
dtype: int64 



In [6]:
#Juntando os três DataFrames em um só

#Ajustando o tamanho das colunas
pd.set_option('max_colwidth', 40)

#Juntando os DataFrames e imprimindo uma amostra dos dados
made_in_brazil_df = pd.DataFrame()
made_in_brazil_df = made_in_brazil_df.append([made_in_brazil_bass_df, made_in_brazil_electric_guitar_df, made_in_brazil_acoustic_guitar_df])

print(made_in_brazil_df.shape)

made_in_brazil_df.sample(5)

(162, 8)


,Name,Brand,Total_Price,Number_Installments,Cash_Price,Image_Url,Description,Store
2,Violão Folk Aço TW 25 Elétrico Dread...,TAGIMA,1069.00,10,962.10,https://madeinbrazil.fbitsstatic.net...,O Tagima TW25 é um violão que tem d...,Made in Brazil
45,Violão Aço Montana Elétrico Série Am...,TAGIMA,1349.00,10,1214.10,https://madeinbrazil.fbitsstatic.net...,O violão Tagima Montana traz um desi...,Made in Brazil
3,Contrabaixo Fretless Vintage Modifie...,SQUIER BY FENDER,3990.00,10,3591.00,https://madeinbrazil.fbitsstatic.net...,"Durante anos, os músicos vem fazendo...",Made in Brazil
7,Contrabaixo Ativo 5 Cordas TBM 5 Sér...,TAGIMA,1999.00,10,1799.10,https://madeinbrazil.fbitsstatic.net...,"O baixo Tagima TBM 5 quatro cordas, ...",Made in Brazil
53,Violão Folk Eletroacústico High Perf...,GIBSON,15790.00,10,14211.00,https://madeinbrazil.fbitsstatic.net...,Esta novidade de 2017 oferece um ino...,Made in Brazil


## 5.0 Barra Music 

### 5.1 Informações do item

In [14]:
def barra_get_item(url_list):
    
    ''' Recebe uma lista de urls de itens do site Barra Music e retorna uma lista de dicionários 
    com as informações de cada instrumento. O resultado pode ser facilmente convertido em um DataFrame'''
    
    item_dict_list = []
    
    for url in url_list:
          
        #Carregando a página e convertendo para um objeto Beautiful Soup    
        r = requests.get(url)
        instrument = bs(r.content)

        item_dict = {}

        #1 de 8 - Nome
        item_dict["Name"] = instrument.find("div", {"class":"box-nome"}).h1.get_text()
        
        #2 de 8 - Marca
        #Checando se existe a informação de marca
        if instrument.find("li", {"class":"txt-marca"}).a == None:
            item_dict["Brand"] = "NaN"
            
        else:
            item_dict["Brand"] = instrument.find("li", {"class":"txt-marca"}).a.get_text()

        #3 de 8 - Preço total
        #Checando se existe a informação de preço
        if instrument.find("li", {"class":"txt-disponibilidade"}).get_text() != "":
            item_dict["Total_Price"] = "NaN"
        
        else:
            total_price = instrument.find("span", {"class":"preco-por"}).span.get_text()
            total_price = total_price.replace("R$","").replace('.','').replace(',','.').strip()
            item_dict["Total_Price"] = total_price

        #4 de 8 Número máximo de parcelas
        #Checando se existe a informação de número de parcelas
        if instrument.find("li", {"class":"txt-disponibilidade"}).get_text() != "":
            item_dict["Number_Installments"] = "NaN"
            
        else:
            item_dict["Number_Installments"] = instrument.find("span", {"class":"parcelamento"}).span.get_text()[:-1]
       
        #5 de 8 Preço à vista
        #Checando se existe a informação de preço à vista
        if instrument.find("li", {"class":"txt-disponibilidade"}).get_text() != "":
            item_dict["Cash_Price"] = "NaN"
            
        else:
            cash_price = instrument.find("span", {"class":"a-vista"}).span.get_text()
            cash_price = cash_price.replace("R$","").replace('.','').replace(',','.').strip()
            item_dict["Cash_Price"] = cash_price

        #6 de 8 URL da imagem do produto
        item_dict["Image_Url"] = instrument.find("div", {"id":"slider"}).a.get("href")
        
        #7 de 8 Descrição do produto
        item_dict["Description"] = instrument.find("div", {"class":"box-descricao-curta"}).get_text().strip()
            
        #8 de 8 Nome da loja
        item_dict["Store"] = instrument.find("li", {"data-id":"26"}).get_text().split(" ")[1]

        #Adicionando o dicionário à lista
        item_dict_list.append(item_dict)
        
    return item_dict_list

### 5.2 Buscando as URLs 

**Temos novidades nesta página ._.** <br>

Abrindo a página usando `request` consegui apenas os 15 primeiros instrumentos, exibição padrão da página. Para exibir mais instrumentos é preciso selecionar manualmente no filtro da página porém **a url não é atualizada**, ou seja, não conseguimos obter todos os instrumentos de determinada categoria utilizando apenas Beautiful Soup. =/

Contei novamente com o apoio do **Selenium** para que primeiro fosse alterado o filtro de exibição para `Todos` mostrando todos os instrumentos, e só em seguida realizar a extração das URLs uma vez que o HTML tenha sido atualizado 

Um detalhe interessante, tentei localizar o menu através da `class` e não estava retornando nenhuma informação. O jeito foi usar o `Xpath` do menu para direcionar especificamente para aquele campo e fazer as alterações. Aí deu certo! 

In [15]:
def barra_get_urls(pages_url):
    
    '''Recebe uma url de categoria como 'https://www.barramusic.com.br/categoria/cordas-guitarras/35143'
    e retorna as urls dos produtos de todas as páginas daquela categoria'''
    
    url_list = []
    
    #Carrega a página
    driver = webdriver.Chrome(executable_path ="chromedriver", options = op)
    driver.get(pages_url)

    #Localiza o filtro de itens por página
    search = driver.find_element_by_xpath('''//*[@id="pagina-listaprodutos"]/div[2]/div/div[2]/div/span/select''')

    #Digita "T" e depois "Enter" 
    search.send_keys("T")
    search.send_keys(Keys.RETURN)

    #Espera 5 segundos para carregar completamente
    time.sleep(5)

    #Código fonte da página que será enviado ao Beautiful Soup
    page_source = driver.page_source
    
    #Convertendo para um objeto Beautiful Soup
    instrument_url = bs(page_source, 'lxml')

    #Número total de instrumentos
    total_links = len(instrument_url.find_all("div", {"class": "item-action"}))
    print("Total instruments: {}".format(total_links))
    
    #Imprime a página atual
    print(pages_url)

    #Percorre a página extraindo as urls dos produtos
    for i in instrument_url.find_all("div", {"class": "item-action"}):
        url_list.append("https://www.barramusic.com.br" + i.a.get("href"))

    print("-----------------")
    
    return url_list

### 5.3 Buscando todos os baixos, guitarras e violões e convertendo para um DataFrame
Aplicando as funções criadas no último site para extrair os dados

In [16]:
#Buscando as urls
barra_bass_urls = barra_get_urls("https://www.barramusic.com.br/categoria/cordas-baixos/35145")
barra_acoustic_guitar_urls = barra_get_urls("https://www.barramusic.com.br/categoria/cordas-violoes/35144")
barra_electric_guitar_urls = barra_get_urls("https://www.barramusic.com.br/categoria/cordas-guitarras/35143")

#Buscando as informações dos itens
barra_bass_info = barra_get_item(barra_bass_urls)
barra_acoustic_guitar_info = barra_get_item(barra_acoustic_guitar_urls)
barra_electric_guitar_info = barra_get_item(barra_electric_guitar_urls)

#Criando o DataFrame
barra_bass_df = pd.DataFrame(barra_bass_info)
barra_acoustic_guitar_df = pd.DataFrame(barra_acoustic_guitar_info)
barra_electric_guitar_df = pd.DataFrame(barra_electric_guitar_info)

Total instruments: 15
https://www.barramusic.com.br/categoria/cordas-baixos/35145
-----------------
Total instruments: 49
https://www.barramusic.com.br/categoria/cordas-violoes/35144
-----------------
Total instruments: 68
https://www.barramusic.com.br/categoria/cordas-guitarras/35143
-----------------


In [17]:
#Checando por valores duplicados
dfs = [barra_bass_df, barra_acoustic_guitar_df, barra_electric_guitar_df]

for i in dfs:
    print(i.shape)
    print(i.duplicated().value_counts(), "\n")

(15, 8)
False    15
dtype: int64 

(49, 8)
False    49
dtype: int64 

(68, 8)
False    68
dtype: int64 



In [18]:
#Juntando os três DataFrames em um só

#Ajustando o tamanho das colunas
pd.set_option('max_colwidth', 40)

#Juntando os DataFrames e imprimindo uma amostra dos dados
barra_df = pd.DataFrame()
barra_df = barra_df.append([barra_bass_df, barra_electric_guitar_df, barra_acoustic_guitar_df])

print(barra_df.shape)

barra_df.sample(5)

(132, 8)


,Name,Brand,Total_Price,Number_Installments,Cash_Price,Image_Url,Description,Store
46,Guitarra Tagima T-805 | Strato | HSS...,Tagima,3149.10,12,2755.46,https://cdn1.appsisecommerce.com.br/...,Um instrumento realmente distinto pa...,Barramusic
27,Guitarra Cort X100 | HH | Open Pore ...,Cort,1529.15,12,1338.01,https://cdn1.appsisecommerce.com.br/...,A Guitarra Cort X100 faz parte da sé...,Barramusic
8,Baixo Fender Squier Classic Vibe 70s...,Fender,3999.00,12,3499.12,https://cdn1.appsisecommerce.com.br/...,O Squier Classic Vibe 70's Precision...,Barramusic
55,Guitarra Vintage V52MR | Tele | Reli...,Guitarras Vintage,3059.10,12,2676.71,https://cdn1.appsisecommerce.com.br/...,Instrumento para o músico que não ab...,Barramusic
38,Violão Cort CEC 1 | Eletro Acustico ...,Cort,1206.15,12,1055.38,https://cdn1.appsisecommerce.com.br/...,Violão Cort CEC 1 Eletro Acústico co...,Barramusic


## 6.0 Exportando para Excel

Pronto! Com as informações desejadas devidamente salvas em um Dataframe, vamos agora salvar os dados em um arquivo Excel. A parte de exploração de dados será realizada na **parte 02**. 

In [7]:
#Exportando para Excel - Tango
tango_df.to_excel(r'Tango_df.xlsx', index = False)

In [7]:
#Exportando para Excel - Made In Brazil
made_in_brazil_df.to_excel(r'Made in Brazil_df.xlsx', index = False)

In [19]:
#Exportando para Excel - Barra Music
barra_df.to_excel(r'Barra_df.xlsx', index = False)